<a href="https://colab.research.google.com/github/Maleniski/ing-caracteristicas/blob/main/Proyecto1_DescargandoDatos_MaElenaMtzMzn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto 1: Descarga de datos

**Estudiante:** María Elena Martínez Manzanares

**Asignatura:** Ingeniería de características

**Indicación:**

1. Un script o libreta que descargue datos de al menos dos fuentes diferentes, y que genere un archivo texto con la descripción de las fuentes, las fechas de descarga y de ser posible la descripción (o enlaces) que expliquen la naturaleza de los datos descargados. Si los datos venían sin explicación, agregar la explicación propia para simplificar el proceso.
2. Una libreta o script que transforme y utilice los datos de acuerdo a su tipo, selecciones la información que se desea utilizar y se generen los Dataframes necesarios.
3. Un diccionario de datos por cada dataframe

El objetivo de este trabajo es obtener información general respecto a Estados Unidos que tienen relación con el estado de Sonora. Se localizaron cuatro fuentes de información que dan datos de diferente naturaleza de donde se reconocen los siguientes cortes diferenciadores.

* [Proyecto Migrantes Desaparecidos](https://missingmigrants.iom.int/es/descargar-datos-del-proyecto-migrantes-desaparecidos) y [Arizona OpenGIS Initiative for Deceased Migrants](https://humaneborders.info/) presentan datos relacionados de muertes de migrantes tratando de cruzar la frontera Sonora-Estados Unidos.

* La [CONAPO](https://datos.gob.mx/busca/dataset/indices-de-intensidad-migratoria-mexico-estados-unidos) presentan datos de intensidad migratoria a nivel estatal y federal.

* Finalmente, los datos provistos por la [SEGOB](http://portales.segob.gob.mx/es/PoliticaMigratoria/Bases_de_datos) presentan información de repatriados desde Estados Unidos a México.

Cada fuente de datos contiene de una a seis tablas. En la siguiente tabla se muestra la agregación de datos realizada de manera más específica.


| Fuente de datos                                 | Cantidad de tablas originales | Cantidad de tablas tidy por fuente |
|-------------------------------------------------|-------------------------------|------------------------------------|
| CONAPO                                          | 3                             | 1                                  |
| SEGOB                                           | 6                             | 1                                  |
| Proyecto Migrantes Desaparecidos                | 5                             | 1                                  |
| Arizona OpenGIS Initiative for Decease Migrants | 1                             | 1                                  |



In [1]:
import pandas as pd
import numpy as np
import datetime
import urllib.request
import os
import zipfile

In [2]:
print(os.getcwd())
subdir = "./data/"

/content


In [3]:
#URL datos de la CONAPO
CONAPO_2000='http://www.conapo.gob.mx/work/models/OMI/Datos_Abiertos/IIM/indice_de_Intensidad_Migratoria_2000.zip'
CONAPO_2000_2='http://www.conapo.gob.mx/work/models/OMI/Datos_Abiertos/DA_IAIM/IAIM_Entidad_2000.csv'
CONAPO_2020='http://www.conapo.gob.mx/work/models/OMI/Datos_Abiertos/IIM/indice_de_Intensidad_Migratoria_2020.zip'

#Ubicacion para datos de la CONAPO
CONAPO_2000_archivo='indice_de_Intensidad_Migratoria_2000.zip'
CONAPO_2000_2_archivo='IAIM_Entidad_2000.csv'
CONAPO_2020_archivo='indice_de_Intensidad_Migratoria_2020.zip'

#URL datos de la SEGOB
SEGOB_2021='http://portales.segob.gob.mx/work/models/PoliticaMigratoria/CEM/Estadisticas/BDatos/DevMexEUU/DwnldBD.php?f=BDSIOMR_PUB_2021_UPMRIP.csv'
SEGOB_2020='http://portales.segob.gob.mx/work/models/PoliticaMigratoria/CEM/Estadisticas/BDatos/DevMexEUU/DwnldBD.php?f=BDSIOMR_PUB_2020_UPMRIP.csv'
SEGOB_2019='http://portales.segob.gob.mx/work/models/PoliticaMigratoria/CEM/Estadisticas/BDatos/DevMexEUU/DwnldBD.php?f=BDSIOMR_PUB_2019_UPMRIP.csv'
SEGOB_2018='http://portales.segob.gob.mx/work/models/PoliticaMigratoria/CEM/Estadisticas/BDatos/DevMexEUU/DwnldBD.php?f=BDSIOMR_PUB_2018_UPMRIP.csv'
SEGOB_2017='http://portales.segob.gob.mx/work/models/PoliticaMigratoria/CEM/Estadisticas/BDatos/DevMexEUU/DwnldBD.php?f=BDSIOMR_PUB_2017_UPMRIP.csv'
SEGOB_2016='http://portales.segob.gob.mx/work/models/PoliticaMigratoria/CEM/Estadisticas/BDatos/DevMexEUU/DwnldBD.php?f=BDSIOMR_PUB_2016_UPMRIP.csv'
Catalogos_SEGOB='http://portales.segob.gob.mx/work/models/PoliticaMigratoria/CEM/Estadisticas/BDatos/DevMexEUU/CatRepatriados_2021.xls'

#Ubicacion para datos de la SEGOB
SEGOB_2021_archivo='BDSIOMR_PUB_2021_UPMRIP.csv'
SEGOB_2020_archivo='BDSIOMR_PUB_2020_UPMRIP.csv'
SEGOB_2019_archivo='BDSIOMR_PUB_2019_UPMRIP.csv'
SEGOB_2018_archivo='BDSIOMR_PUB_2018_UPMRIP.csv'
SEGOB_2017_archivo='BDSIOMR_PUB_2017_UPMRIP.csv'
SEGOB_2016_archivo='BDSIOMR_PUB_2016_UPMRIP.csv'
Catalogos_SEGOB_archivo="Catalogo_SEGOB.xls"

#URL datos del Proyecto Migrantes Desaparecidos (PMD)
PMD_2020='https://missingmigrants.iom.int/global-figures/2020/csv'
PMD_2017='https://missingmigrants.iom.int/global-figures/2017/csv'
PMD_2016='https://missingmigrants.iom.int/global-figures/2016/csv'
PMD_2015='https://missingmigrants.iom.int/global-figures/2015/csv'
PMD_2014='https://missingmigrants.iom.int/global-figures/2014/csv'

#Ubicacion para datos del PMD
PMD_2020_archivo='pmd2020.csv'
PMD_2017_archivo='pmd2017.csv'
PMD_2016_archivo='pmd2016.csv'
PMD_2015_archivo='pmd2015.csv'
PMD_2014_archivo='pmd2014.csv'

#URL datos del Arizona OpenGIS Initiative for Deceased Migrants
ARIZONA_DM='https://humaneborders.info/app/getTableByMultipleSearch.asp?sex=&cod=&county=&corridors=&sm=&years=&name=&detail=full&format=csv'

#Ubicacion para datos del DM
ARIZONA_DM_archivo='DM.csv'

In [4]:
CONAPO = { CONAPO_2000:CONAPO_2000_archivo, CONAPO_2000_2:CONAPO_2000_2_archivo, CONAPO_2020:CONAPO_2020_archivo }
SEGOB = { SEGOB_2021:SEGOB_2021_archivo, SEGOB_2020:SEGOB_2020_archivo, 
        SEGOB_2019:SEGOB_2019_archivo, SEGOB_2018:SEGOB_2018_archivo, 
        SEGOB_2017:SEGOB_2017_archivo, SEGOB_2016:SEGOB_2016_archivo,
         Catalogos_SEGOB: Catalogos_SEGOB_archivo}
PMD = { PMD_2020:PMD_2020_archivo, PMD_2017:PMD_2017_archivo,
        PMD_2016:PMD_2016_archivo, PMD_2015:PMD_2015_archivo,
        PMD_2014:PMD_2014_archivo }
ARIZONA = { ARIZONA_DM:ARIZONA_DM_archivo }

In [31]:
for url, archivo in CONAPO.items(): 
    if not os.path.exists(archivo):
        if not os.path.exists(subdir):
            os.makedirs(subdir)
        urllib.request.urlretrieve(url, subdir + archivo)  
        if(url != CONAPO_2000_2):
            with zipfile.ZipFile(subdir + archivo, "r") as zip_ref:
                zip_ref.extractall(subdir)
print("Descarga de CONAPO terminada.")

for url, archivo in SEGOB.items(): 
    if not os.path.exists(archivo):
        if not os.path.exists(subdir):
            os.makedirs(subdir)
        urllib.request.urlretrieve(url, subdir + archivo)
print("Descarga de SEGOB terminada.")

for url, archivo in PMD.items(): 
    if not os.path.exists(archivo):
        if not os.path.exists(subdir):
            os.makedirs(subdir)
        urllib.request.urlretrieve(url, subdir + archivo)
print("Descarga de Proyecto Migrantes Desaparecidos terminada.")
          
    
for url, archivo in ARIZONA.items(): 
    if not os.path.exists(archivo):
        if not os.path.exists(subdir):
            os.makedirs(subdir)
        urllib.request.urlretrieve(url, subdir + archivo)
print("Descarga de Arizona OpenGIS Initiative for Deceased Migrants.")          

with open(subdir + "info.txt", 'w') as f:
    f.write("Archivos con datos que relaciona Estados Unidos con Sonora. \n")
    info = """
    Los datos fueron descargados de los portales de la CONAPO, SEGOB, Proyecto Migrantes Desaparecidos y 
    el Arizona OpenGIS Initiative for Deceased Migrants. Los datos presentan información de diferente naturaleza
    donde se reconocen los siguientes cortes diferenciadores.

    Proyecto Migrantes Desaparecidos y Arizona OpenGIS Initiative for Deceased Migrants presentan datos 
    relacionados de muertes de migrantes tratando de cruzar la frontera Sonora-Estados Unidos.

    La CONAPO presentan datos de intensidad migratoria a nivel estatal y federal.

    Finalmente, los datos provistos por la SEGOB presentan información de repatriados desde Estados Unidos
    a México.

    """ 
    f.write(info + '\n')
    f.write("Descargado el " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
    f.write("|-------------------------Metadatos de la descarga de la CONAPO-------------------------|"+'\n')
    for url, archivo in CONAPO.items():        
        f.write("Desde: " + url + "\n")
        f.write("Nombre: " + archivo + "\n")
    f.write("|-------------------------Metadatos de la descarga de la SEGOB-------------------------|"+'\n')    
    for url, archivo in SEGOB.items():        
        f.write("Desde: " + url + "\n")
        f.write("Nombre: " + archivo + "\n")
    f.write("|-------------Metadatos de la descarga del Proyecto Migrantes Desaparecidos------------|"+'\n')    
    for url, archivo in PMD.items():        
        f.write("Desde: " + url + "\n")
        f.write("Nombre: " + archivo + "\n")              
    f.write("|-------------Metadatos de la descarga del Arizona OpenGIS Initiative for Deceased Migrants------------|"+'\n') 
    for url, archivo in ARIZONA.items():        
        f.write("Desde: " + url + "\n")
        f.write("Nombre: " + archivo + "\n")  

Descarga de CONAPO terminada.
Descarga de SEGOB terminada.
Descarga de Proyecto Migrantes Desaparecidos terminada.
Descarga de Arizona OpenGIS Initiative for Deceased Migrants.


# Creación de tabla tidy a partir de los datos del Arizona OpenGIS Initiative for Deceased Migrants

La tabla de datos descargada de esta fuente presentan un registro de localizacion de cadáveres encontrados en Arizona. Principalmente se tiene registro del lugar y fecha de ubicación, estado de descomposición y posibles causas de la muerte.

## Limpieza.

In [6]:
arizona=pd.read_csv("data//"+ARIZONA_DM_archivo)

In [7]:
arizona_columns=['Name', 
                'Sex', 
                'Age', 
                'Reporting Date',
                'Cause of Death',
                'OME Determined COD',
                'Body Condition',
                'Post Mortem Interval',  
                'County']

ARIZONA_tidy=arizona[arizona_columns].copy()
ARIZONA_tidy['Reporting Date'] = ARIZONA_tidy['Reporting Date'].astype('datetime64[ns]')
ARIZONA_tidy['Age'] = ARIZONA_tidy['Age'].astype('Int64')

## Tabla.

In [8]:
ARIZONA_tidy

,Name,Sex,Age,Reporting Date,Cause of Death,OME Determined COD,Body Condition,Post Mortem Interval,County
0,Unidentified,male,<NA>,2000-01-10,Skeletal Remains,UNDETERMINED - MUMMIFIED AND SKELETAL REMAINS,Skeletonization w/ mummification,< 5 weeks,YUMA
1,"Nicolas, Tomas",male,17,2000-01-22,Exposure,HYPOTHERMIA,Fully fleshed,< 1 day,PIMA
2,Unidentified,male,<NA>,2000-01-20,Skeletal Remains,UNDETERMINED - SKELETAL REMAINS,Complete skeletonization w/ bone degradation,> 6-8 months,SANTA CRUZ
3,Unidentified,male,<NA>,2000-01-10,Skeletal Remains,UNDETERMINED - SKELETAL REMAINS,Complete skeletonization w/ disarticulation,< 6-8 months,PIMA
4,"Candia-Bravo, Maria Del Rocio",female,20,2000-02-05,Blunt Force Injury,MULTIPLE BLUNT FORCE INJURIES,Fully fleshed,< 1 day,PIMA
...,...,...,...,...,...,...,...,...,...
3943,Unidentified,undetermined,<NA>,1994-02-09,Undetermined,Undetermined,NaN,NaN,Maricopa
3944,Jose Alvarado,male,<NA>,1995-08-15,Gunshot Wound,Multiple gunshot wounds,NaN,NaN,Maricopa
3945,Jose Espinoza,male,<NA>,1995-08-15,Gunshot Wound,Multiple gunshot wounds,NaN,NaN,Maricopa
3946,Unidentified,male,<NA>,1996-02-27,Undetermined,Undetermined,NaN,NaN,Maricopa


## Diccionario.

| Variable             | Tipo de etiqueta | Etiqueta                                       | Descriptor de la variable                                                               | Tipo de variable     | Unidades     |
|----------------------|------------------|------------------------------------------------|-----------------------------------------------------------------------------------------|----------------------|--------------|
| Name                 | Nominal          | Nombre                                         | Nombre de la persona ubicada sin vida.                                                  | String               | -            |
| Sex                  | Nominal          | Sexo                                           | Sexo de la persona ubicada sin vida.                                                    | String (male/female) | -            |
| Age                  | Ordinal          | Edad                                           | Edad de la persona ubicada sin vida.                                                    | Int                  | [0,$\infty$) |
| Reporting Date       | Ordinal          | Fecha de reporte                               | Fecha en que se reportó el avistamiento del cuerpo.                                     | Datetime             | Año-mes-día  |
| Cause of Death       | Nominal          | Motivos de la muerte                           | Descripción breve de las causas de la muerte.                                           | String               |              |
| OME Determined COD   | Nominal          | Causa de la muerte determinada por el forense. | Causa de la muerte (COD) determinada por el forense (Office of Medical Examiner / OME). | String               |              |
| Body Condition       | Nominal          | Condiciones del cuerpo                         | Nivel de putrefacción del cuerpo al momento de ser encontrado.                          | String               |              |
| Post Mortem Interval | Ordinal          | Intervalo después de la muerte.                | Una estimación en meses del tiempo que llevaba la persona fallecida.                    | String               |              |
| County               | Nominal          | Condado.                                       | Condado (equivalente a municipio) donde fue encontrado el cuerpo.                       | String               |              |

# Creación de tabla tidy a partir de los datos de la CONAPO.

Datos de índices de Intensidad Migratoria México-Estados Unidos, se restringen las tablas a datos relacionados con Sonora.

## Limpieza.

In [9]:
CONAPO1=pd.read_csv("data//IAIM_Entidad_2000.csv", encoding='latin-1')
CONAPO2=pd.read_csv("data//╓ndice de Intensidad Migratoria 2000//iim_base2000e.csv")
CONAPO3=pd.read_csv("data//╓ndice de Intensidad Migratoria 2000//iim_base2000m.csv")
CONAPO4=pd.read_csv("data//╓ndice de Intensidad Migratoria 2020//iim_base2020e.csv")
CONAPO5=pd.read_csv("data//╓ndice de Intensidad Migratoria 2020//iim_base2020m.csv", encoding='latin-1')
estados_sonora=CONAPO3[CONAPO3["ENT"]==26]["NOM_MUN"].tolist()

CONAPO5=CONAPO5[CONAPO5['NOM_MUN'].isin(estados_sonora)].tail(72)
columnas = CONAPO2[CONAPO2["ENT"]==26].columns.tolist()

In [10]:
columnas.insert(1, "YEAR")
columnas.insert(2, "NOM_MUN")
columnas.insert(len(columnas), "LUGAR")

columnas.remove("ENT")

CONAPO_tidy=(pd.DataFrame(columns=columnas)
            .rename(columns={"IIM_2000":"IIM", "GIM_2000":"GIM"})
            .drop(columns=['NOM_ENT']))
CONAPO_tidy

,YEAR,NOM_MUN,TOT_HOG,HOG_REM,HOG_REM.1,HOG_CIRC,HOG_RET,IIM,GIM,LUGAR


In [11]:
CONAPO_tidy = pd.concat([CONAPO_tidy, pd.DataFrame.
                        from_records([{'YEAR': 2000,
                                        "NOM_MUN": "Estado",
                                        "TOT_HOG":CONAPO2[CONAPO2["ENT"]==26]["TOT_HOG"].iloc[0],
                                        "HOG_REM":CONAPO2[CONAPO2["ENT"]==26]["HOG_REM"].iloc[0],
                                        "HOG_REM.1":CONAPO2[CONAPO2["ENT"]==26]["HOG_REM.1"].iloc[0],
                                        "HOG_CIRC":CONAPO2[CONAPO2["ENT"]==26]["HOG_CIRC"].iloc[0],
                                        "HOG_RET":CONAPO2[CONAPO2["ENT"]==26]["HOG_RET"].iloc[0],
                                        "IIM":CONAPO2[CONAPO2["ENT"]==26]["IIM_2000"].iloc[0],
                                        "GIM":CONAPO2[CONAPO2["ENT"]==26]["GIM_2000"].iloc[0],
                                        "LUGAR":np.nan}])])
CONAPO_tidy

,YEAR,NOM_MUN,TOT_HOG,HOG_REM,HOG_REM.1,HOG_CIRC,HOG_RET,IIM,GIM,LUGAR
0,2000,Estado,539528,3.159984,1.586943,0.318241,0.869464,-0.639287,Bajo,NaN


In [12]:
for registro in range(CONAPO3[CONAPO3["ENT"]==26].shape[0]):
    CONAPO_tidy = pd.concat([CONAPO_tidy, pd.DataFrame.
                            from_records([{'YEAR': 2000,
                                            "NOM_MUN":CONAPO3[CONAPO3["ENT"]==26]["NOM_MUN"].iloc[registro],
                                            "TOT_HOG":CONAPO3[CONAPO3["ENT"]==26]["TOT_HOG"].iloc[registro],
                                            "HOG_REM":CONAPO3[CONAPO3["ENT"]==26]["HOG_REM"].iloc[registro],
                                            "HOG_REM.1":CONAPO3[CONAPO3["ENT"]==26]["HOG_REM.1"].iloc[registro],
                                            "HOG_CIRC":CONAPO3[CONAPO3["ENT"]==26]["HOG_CIRC"].iloc[registro],
                                            "HOG_RET":CONAPO3[CONAPO3["ENT"]==26]["HOG_RET"].iloc[registro],
                                            "IIM":CONAPO3[CONAPO3["ENT"]==26]["IIM_2000"].iloc[registro],
                                            "GIM":CONAPO3[CONAPO3["ENT"]==26]["GIM_2000"].iloc[registro],
                                            "LUGAR":np.nan}])])  

In [13]:
for registro in range(CONAPO4[CONAPO4["CVE_ENT"]==26].shape[0]):
    CONAPO_tidy = pd.concat([CONAPO_tidy, pd.DataFrame.
                            from_records([{'YEAR': 2020,
                                            "NOM_MUN": "Estado",
                                            "TOT_HOG":CONAPO4[CONAPO4["CVE_ENT"]==26]["VIV_TOT"].iloc[registro],
                                            "HOG_REM":CONAPO4[CONAPO4["CVE_ENT"]==26]["VIV_REM"].iloc[registro],
                                            "HOG_REM.1":CONAPO4[CONAPO4["CVE_ENT"]==26]["VIV_EMIG"].iloc[registro],
                                            "HOG_CIRC":CONAPO4[CONAPO4["CVE_ENT"]==26]["VIV_CIRC"].iloc[registro],
                                            "HOG_RET":CONAPO4[CONAPO4["CVE_ENT"]==26]["VIV_RET"].iloc[registro],
                                            "IIM":CONAPO4[CONAPO4["CVE_ENT"]==26]["IIM-DP2"].iloc[registro],
                                            "GIM":CONAPO4[CONAPO4["CVE_ENT"]==26]["GIM-DP2"].iloc[registro],
                                            "LUGAR":CONAPO4[CONAPO4["CVE_ENT"]==26]["LUGAR"].iloc[registro]}])])  

In [14]:
for registro in range(CONAPO5.shape[0]):
    CONAPO_tidy = pd.concat([CONAPO_tidy, pd.DataFrame.
                            from_records([{'YEAR': 2020,
                                            "NOM_MUN":CONAPO5["NOM_MUN"].iloc[registro],
                                            "TOT_HOG":CONAPO5["VIV_TOT"].iloc[registro],
                                            "HOG_REM":CONAPO5["VIV_REM"].iloc[registro],
                                            "HOG_REM.1":CONAPO5["VIV_EMIG"].iloc[registro],
                                            "HOG_CIRC":CONAPO5["VIV_CIRC"].iloc[registro],
                                            "HOG_RET":CONAPO5["VIV_RET"].iloc[registro],
                                            "IIM":CONAPO5["IIM-DP2"].iloc[registro],
                                            "GIM":CONAPO5["GIM-DP2"].iloc[registro],
                                            "LUGAR":CONAPO5["LUGAR"].iloc[registro]}])])   

In [15]:
CONAPO_tidy.reset_index(drop=True,inplace=True)
CONAPO_tidy['YEAR'] = CONAPO_tidy['YEAR'].astype('int')

## Tabla.

In [16]:
CONAPO_tidy

,YEAR,NOM_MUN,TOT_HOG,HOG_REM,HOG_REM.1,HOG_CIRC,HOG_RET,IIM,GIM,LUGAR
0,2000,Estado,539528,3.159984,1.586943,0.318241,0.869464,-0.639287,Bajo,NaN
1,2000,Aconchi,578,2.249135,4.325260,0.173010,0.519031,-0.489189,Bajo,NaN
2,2000,Agua Prieta,14876,2.742673,3.267007,0.134445,2.171283,-0.246991,Bajo,NaN
3,2000,Alamos,5887,2.429081,2.310175,0.322745,0.373705,-0.579205,Bajo,NaN
4,2000,Altar,1690,3.668639,0.532544,0.000000,0.650888,-0.599683,Muy bajo,NaN
...,...,...,...,...,...,...,...,...,...,...
141,2020,Benito Juárez,4581,1.746344,0.567562,0.065488,0.000000,64.996600,Muy bajo,2143.0
142,2020,Magdalena,853,0.468933,0.117233,0.000000,0.000000,65.307900,Muy bajo,2428.0
143,2020,Nogales,10464,3.307839,0.382263,0.047861,0.172084,64.824370,Muy bajo,1956.0
144,2020,Benito Juárez,1276,18.416928,5.799373,0.315706,3.134796,60.972490,Alto,385.0


## Diccionario.

| Variable  | Tipo de etiqueta | Etiqueta                          | Descriptor de la variable                                                                                                                                                                                                                                              | Tipo de variable | Unidades                                    |
|-----------|------------------|-----------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------------|---------------------------------------------|
| YEAR      | Nominal          | Año                               | Año del cálculo del índice de intensidad migratoria.                                                                                                                                                                                                                   | Int              | 2000-2020                                   |
| NOM_MUN   | Nominal          | Nombre del municipio              | Nombre del municipio. El municipio "Estado" hace referencia a que el renglón o registro en cuestion presenta indicadores a nivel estado.                                                                                                                               | String           |                                             |
| TOT_HOG   | Ordinal          | Total de hogares.                 | Total de hogares en el municipio (o estado).                                                                                                                                                                                                                           | Int              | [0,$\infty$)                                |
| HOG_REM   | Ordinal          | Hogares que reciben remesas       | Porcentaje de hogares que reciben remesas en el municipio (o estado).                                                                                                                                                                                                  | Float            | [0,100]                                     |
| HOG_REM.1 | Ordinal          | Hogares con emigrantes en EUA     | Porcentaje de hogares con emigrantes en Estados Unidos del quinquenio anterior en el municipio (o estado).                                                                                                                                                             | Float            | [0,100]                                     |
| HOG_CIRC  | Ordinal          | Hogares con emigrantes circulares | Porcentaje de hogares con migrantes circulares del quinquenio anterior en el municipio (o estado).                                                                                                                                                                     | Float            | [0,100]                                     |
| HOG_RET   | Ordinal          | Hogares con migrantes de retorno  | Porcentaje de hogares con migrantes de retorno del quinenio anterior en el municipio (o estado).                                                                                                                                                                       | Float            | [0,100]                                     |
| IIM       | Ordinal          | Índice de intensidad migratoria   | Índice de intensidad migratoria.                                                                                                                                                                                                                                       | Float            | ($-\infty$, $\infty$)                       |
| GIM       | Ordinal          | Grado de intensidad migratoria    | Grado de intensidad migratoria                                                                                                                                                                                                                                         | String           | Bajo, Muy bajo, Medio, Muy alto, Nulo, Alto |
| LUGAR     | Ordinal          | Lugar a nivel nacional.           | Posición que ocupa la entidad a escala nacional en el año en cuestión. Si es el municipio "Estado" hace referencia a la escala por estados, en caso contrario, es la escala por municipio de todos los municipios de México. Cálculado a partir de registros del 2020. | Int              | [1, 2458]                                   |

# Creación de tabla tidy a partir de los datos del Proyecto Migrantes Desaparecidos.

Cada registro de los datos descargados de esta fuente tienen información a nivel mundial de eventos con fatalidades o lesionados de migrantes durante su trayecto. Estos eventos con fatalidades puede ser la localización de un cuerpo de un migrante fallecido en su intento por trasladarse.

Se restringe la información mundial a la que tenga relación con Arizona. Estos datos extiende la información descargada Arizona OpenGIS Initiative for Deceased Migrants ya que en un solo registro puede añadirse eventos donde murieron más de una persona, así como también añade información de si en un evento hubieron sobrevivientes.


## Limpieza.

In [17]:
PMD1=pd.read_csv("data//" + PMD_2014_archivo)
PMD1=PMD1[PMD1["Location of death"].str.contains("Arizona")]

PMD2=pd.read_csv("data//" + PMD_2015_archivo)
PMD2=PMD2[PMD2["Location of death"].str.contains("Arizona")]

PMD3=pd.read_csv("data//" + PMD_2016_archivo)
PMD3=PMD3[PMD3["Location of death"].str.contains("Arizona")]

PMD4=pd.read_csv("data//" + PMD_2017_archivo)
PMD4=PMD4[PMD4["Location of death"].str.contains("Arizona")]

PMD5=pd.read_csv("data//" + PMD_2020_archivo)
PMD5=PMD5[PMD5["Location of death"].str.contains("Arizona")]

In [18]:
columnas_pmd=["Incident_Date", 
                "Year",
                "Reported_Month",
                "Number_Dead",
                "Total_Number_of_Dead_and_Missing",
                "Number_of_Survivors",
                'Number_of_Females', 
                'Number_of_Males', 
                'Number_of_Children',
                'Cause_of_Death',
                'Location_of_Death']

PMD_tidy=pd.DataFrame(columns=columnas_pmd)

In [19]:
for tabla_pmd in PMD1, PMD2, PMD3, PMD4, PMD5:
  for registro in range(tabla_pmd.shape[0]):
      PMD_tidy = pd.concat([PMD_tidy, pd.DataFrame.
                          from_records([{"Incident_Date": tabla_pmd["Incident Date"].iloc[registro], 
                                          "Year": tabla_pmd["Year"].iloc[registro],
                                          "Reported_Month":tabla_pmd["Reported Month"].iloc[registro],
                                          "Number_Dead":tabla_pmd["Number Dead"].iloc[registro],
                                          "Total_Number_of_Dead_and_Missing":tabla_pmd["Total Number of Dead and Missing"].iloc[registro],
                                          "Number_of_Survivors":tabla_pmd["Number of Survivors"].iloc[registro],
                                          'Number_of_Females':tabla_pmd["Number of Females"].iloc[registro],
                                          'Number_of_Males':tabla_pmd["Number of Males"].iloc[registro],
                                          'Number_of_Children':tabla_pmd["Number of Children"].iloc[registro],
                                          'Cause_of_Death':tabla_pmd["Cause of Death"].iloc[registro],
                                          'Location_of_Death':tabla_pmd["Location of death"].iloc[registro]}])]) 

In [20]:
PMD_tidy.reset_index(drop=True,inplace=True)
PMD_tidy['Year'] = PMD_tidy['Year'].astype('int')
PMD_tidy['Number_Dead'] = PMD_tidy['Number_Dead'].astype('int')

In [21]:
PMD_tidy["Incident_Date"]=(pd.to_datetime(PMD_tidy["Incident_Date"])
                            .dt.strftime('%a, %m/%d/%y - %H:%M'))
PMD_tidy["Incident_Date"]=(pd.to_datetime(PMD_tidy["Incident_Date"],
                                            format='%a, %m/%d/%y - %H:%M'))                            

## Tabla.

In [22]:
PMD_tidy

,Incident_Date,Year,Reported_Month,Number_Dead,Total_Number_of_Dead_and_Missing,Number_of_Survivors,Number_of_Females,Number_of_Males,Number_of_Children,Cause_of_Death,Location_of_Death
0,2014-01-06 12:00:00,2014,January,1,1,NaN,NaN,1.0,NaN,Mixed or unknown,Pima Country Office of the Medical Examiner ju...
1,2014-01-12 12:00:00,2014,January,1,1,NaN,NaN,NaN,NaN,Mixed or unknown,Pima Country Office of the Medical Examiner ju...
2,2014-01-14 12:00:00,2014,January,1,1,NaN,NaN,NaN,NaN,Mixed or unknown,Pima Country Office of the Medical Examiner ju...
3,2014-01-16 12:00:00,2014,January,1,1,NaN,NaN,1.0,NaN,Violence,"near Douglas, Arizona, USA"
4,2014-01-17 12:00:00,2014,January,1,1,NaN,NaN,NaN,NaN,Violence,Pima Country Office of the Medical Examiner ju...
...,...,...,...,...,...,...,...,...,...,...,...
826,2020-06-16 12:00:00,2020,June,1,1,NaN,NaN,1.0,NaN,Violence,Pima Country Office of the Medical Examiner ju...
827,2020-09-14 12:00:00,2020,September,1,1,NaN,NaN,1.0,NaN,Mixed or unknown,Pima Country Office of the Medical Examiner ju...
828,2020-10-16 12:00:00,2020,October,1,1,NaN,NaN,1.0,NaN,Mixed or unknown,Pima Country Office of the Medical Examiner ju...
829,2020-10-16 12:00:00,2020,October,1,1,NaN,NaN,1.0,NaN,Mixed or unknown,Pima Country Office of the Medical Examiner ju...


## Diccionario.

| Variable                         | Tipo de etiqueta | Etiqueta                           | Descriptor de la variable                                                                                                                                                                 | Tipo de variable | Unidades         |
|----------------------------------|------------------|------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------------|------------------|
| Incident_Date                    | Ordinal          | Fecha de incidente.                | Fecha en que se ocurrió el evento de accidente a migrantes ilegales en su trayecto por el cruce por la frontera Sonora-Arizona. Puede ser la fecha de descrubiendo del (los) cadáver(es). | DateTime         | Año-mes-día hora |
| Year                             | Ordinal          | Año del incidente.                 | Año en que ocurrió el incidente.                                                                                                                                                          | Int              | 2014-2020        |
| Reported_Month                   | Nominal          | Mes del incidente.                 | Mes en que se reporta el incidente.                                                                                                                                                       | String           |                  |
| Number_Dead                      | Ordinal          | Número de muertos.                 | Cantidad de muertos que se reporta del incidente.                                                                                                                                         | Int              | [0,$\infty$)     |
| Total_Number_of_Dead_and_Missing | Ordinal          | Número de muertos y desaparecidos. | Cantidad final de personas involucradas en el incidente, ya sea muertas o en calidad de desaparecidas.                                                                                    | Int              | [0, $\infty$)    |
| Number_of_Survivors              | Ordinal          | Número de sobrevivientes.          | Cantidad de personas que sobrevivieron en el incidente.                                                                                                                                   | Int              | [0, $\infty$)    |
| Number_of_Females                | Ordinal          | Número de mujeres.                 | Cantidad de mujeres involucradas en el incidente, ya sea muertas, en calidad de desaparecidas o localizadas.                                                                              | Int              | [0, $\infty$)    |
| Number_of_Males                  | Ordinal          | Número de hombres.                 | Cantidad de hombres involucrados en el incidente, ya sea muertos, en calidad de desaparecidos o localizados.                                                                              | Int              | [0, $\infty$)    |
| Number_of_Children               | Ordinal          | Número de niños.                   | Cantidad de niños involucrados en el incidente, ya sea muertos, en calidad de desaparecidos o localizados.                                                                                | Int              | [0, $\infty$)    |
| Cause_of_Death                   | Nominal          | Causa de la muerte.                | Causa de la muerte de las personas involucradas en el incidente.                                                                                                                          | String           |                  |
| Location_of_Death                | Nominal          | Lugar de la muerte.                | Lugar donde ocurrió el incidente, u Oficina del Forente que examinó los cadáveres.                                                                                                        | String           |                  |

# Creación de tabla tidy a partir de los datos de la SEGOB.

Los datos de la SEGOB presentan información de mexicanos repatriados desde Estados Unidos. Esta información incluye cantidad de veces que han sido repatriados, desde donde fueron capturados, indicadores del tratamiento que recibieron durante su proceso de repatriación así de donde son originarios y a donde se dirigen.

Se restringe la información repatriados que son originarios de Sonora, se dirigen a Sonora, que el punto de repatriación sea Sonora o hayan sido capturados en Arizona.

## Limpieza.

In [23]:
SEGOB1=pd.read_csv("data//BDSIOMR_PUB_2016_UPMRIP.csv")
#Registros que estan relacionados con Sonora o la frontera con Sonora.
SEGOB1=SEGOB1[ (SEGOB1["DEL"]==26) | #Entidad federativa donde se ubica el punto de repatriación (Sonora)
        (SEGOB1["EN_NAC"]==26) | #Entidad federativa de nacimiento de la persona repatriada. (Sonora)
        (SEGOB1["EDO_DET"]==4) | #Estado de la Unión Americana donde la persona repatriada declaró haber sido detenido. (Arizona)
        (SEGOB1["EN_DES"]==26) ] #Entidad federativa declarada por la persona repatriada como el lugar al que se dirige. (Sonora)

SEGOB2=pd.read_csv("data//BDSIOMR_PUB_2017_UPMRIP.csv")
#Registros que estan relacionados con Sonora o la frontera con Sonora.
SEGOB2=SEGOB2[ (SEGOB2["DEL"]==26) | #Entidad federativa donde se ubica el punto de repatriación (Sonora)
        (SEGOB2["EN_NAC"]==26) | #Entidad federativa de nacimiento de la persona repatriada. (Sonora)
        (SEGOB2["EDO_DET"]==4) | #Estado de la Unión Americana donde la persona repatriada declaró haber sido detenido. (Arizona)
        (SEGOB2["EN_DES"]==26) ] #Entidad federativa declarada por la persona repatriada como el lugar al que se dirige. (Sonora)

SEGOB3=pd.read_csv("data//BDSIOMR_PUB_2018_UPMRIP.csv")
#Registros que estan relacionados con Sonora o la frontera con Sonora.
SEGOB3=SEGOB3[ (SEGOB3["DEL"]==26) | #Entidad federativa donde se ubica el punto de repatriación (Sonora)
        (SEGOB3["EN_NAC"]==26) | #Entidad federativa de nacimiento de la persona repatriada. (Sonora)
        (SEGOB3["EDO_DET"]==4) | #Estado de la Unión Americana donde la persona repatriada declaró haber sido detenido. (Arizona)
        (SEGOB3["EN_DES"]==26) ] #Entidad federativa declarada por la persona repatriada como el lugar al que se dirige. (Sonora)

SEGOB4=pd.read_csv("data//BDSIOMR_PUB_2019_UPMRIP.csv")
#Registros que estan relacionados con Sonora o la frontera con Sonora.
SEGOB4=SEGOB4[ (SEGOB4["DEL"]==26) | #Entidad federativa donde se ubica el punto de repatriación (Sonora)
        (SEGOB4["EN_NAC"]==26) | #Entidad federativa de nacimiento de la persona repatriada. (Sonora)
        (SEGOB4["EDO_DET"]==4) | #Estado de la Unión Americana donde la persona repatriada declaró haber sido detenido. (Arizona)
        (SEGOB4["EN_DES"]==26) ] #Entidad federativa declarada por la persona repatriada como el lugar al que se dirige. (Sonora)

SEGOB5=pd.read_csv("data//BDSIOMR_PUB_2020_UPMRIP.csv")
#Registros que estan relacionados con Sonora o la frontera con Sonora.
SEGOB5=SEGOB5[ (SEGOB4["DEL"]==26) | #Entidad federativa donde se ubica el punto de repatriación (Sonora)
        (SEGOB5["EN_NAC"]==26) | #Entidad federativa de nacimiento de la persona repatriada. (Sonora)
        (SEGOB5["EDO_DET"]==4) | #Estado de la Unión Americana donde la persona repatriada declaró haber sido detenido. (Arizona)
        (SEGOB5["EN_DES"]==26) ] #Entidad federativa declarada por la persona repatriada como el lugar al que se dirige. (Sonora)


SEGOB6=pd.read_csv("data//BDSIOMR_PUB_2021_UPMRIP.csv")
#Registros que estan relacionados con Sonora o la frontera con Sonora.
SEGOB6=SEGOB6[ (SEGOB6["DEL"]==26) | #Entidad federativa donde se ubica el punto de repatriación (Sonora)
        (SEGOB6["EN_NAC"]==26) | #Entidad federativa de nacimiento de la persona repatriada. (Sonora)
        (SEGOB6["EDO_DET"]==4) | #Estado de la Unión Americana donde la persona repatriada declaró haber sido detenido. (Arizona)
        (SEGOB6["EN_DES"]==26) ] #Entidad federativa declarada por la persona repatriada como el lugar al que se dirige. (Sonora)  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [24]:
SEGOB_tidy=pd.concat([SEGOB1, SEGOB2, SEGOB3, SEGOB4, SEGOB5, SEGOB6])
SEGOB_tidy.drop(["IER","PUNT_REP"],axis=1,inplace=True)
SEGOB_tidy['FEC_REP'] = SEGOB_tidy['FEC_REP'].astype('datetime64[ns]')

## Tabla.

In [25]:
SEGOB_tidy

,DEL,FEC_REP,CLASIF_REP,IRE,SEXO,EDA,EN_NAC,NIV_ESC,ACOM_REP,PERM_EU,...,APO_TRAS,APO_VES,APO_ACT_NAC,APO_TRASF,APO_CURP,APO_ASF,APO_AME,APO_REC_PERT,APO_AT_MEDICA,MUN_NAC
0,26,2016-06-06,1,16000001,1,100,14,7,3,1,...,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,26,2016-01-22,1,16000002,1,99,21,5,3,1,...,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,26,2016-11-10,1,16000003,1,100,12,7,3,1,...,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
6,26,2016-07-10,1,16000007,1,88,14,2,3,1,...,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
9,26,2016-06-03,1,16000010,1,83,14,2,3,4,...,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160675,26,2021-12-31,1,VAOALF11270736640015000000,1,36,15,3,3,1,...,0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,15058.0
160676,26,2021-12-31,1,VEHERIA1130648896001100000,1,25,11,7,3,1,...,0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,11023.0
160677,26,2021-12-31,1,CUCJUAE1130670496001100000,1,25,11,3,3,1,...,0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,11001.0
160681,26,2021-12-31,1,ZAQREB21306316160020000000,2,25,20,5,4,1,...,0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,20526.0


## Diccionario.

| Variable      | Tipo de etiqueta | Etiqueta                                                   | Descriptor de la variable                                                                                                                                                                                                                         | Tipo de variable | Unidades                               |
|---------------|------------------|------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------------|----------------------------------------|
| DEL           | Nominal          | Oficina del INM.                                           | Entidad federativa donde se ubica el punto de repatriación.                                                                                                                                                                                       | Int              | 2, 6, 7,9,10,14,16,21,22,26,27,29,9999 |
| FEC_REP       | Ordinal          | Fecha de repatriación.                                     | Fecha en la cual se registró el evento de repatriación.                                                                                                                                                                                           | DateTime         | Año-mes-día                            |
| CLASIF_REP    | Nominal          | Tipo de evento de repatriación.                            | Clasifica los eventos de repatriación según las reincidencias de devolución.                                                                                                                                                                      | Int              | 1-3                                    |
| IRE           | Nominal          | Identificador clave.                                       | Clave única para identificar las incidencias de repatriación de una misma persona.                                                                                                                                                                | String           |                                        |
| SEXO          | Nominal          | Sexo.                                                      | Sexo del repatriado.                                                                                                                                                                                                                              | Bool             | 1-2                                    |
| EDA           | Ordinal          | Edad.                                                      | Edad de la persona repatriada.                                                                                                                                                                                                                    | Int              | [0,$\infty$)                           |
| EN_NAC        | Nominal          | Entidad de nacimiento.                                     | Entidad federativa de nacimiento de la persona repatriada.                                                                                                                                                                                        | Int              | 1-32, 9999                             |
| NIV_ESC       | Nominal          | Nivel máximo de estudios.                                  | Hace referencia al nivel máximo de estudios declarado por la persona devuelta o el acompañante en el caso de niñas, niños o adolescentes.                                                                                                         | Int              | 1-11, 9999                             |
| ACOM_REP      | Nominal          | Condición de acompañamiento del repatriado.                | Hace referencia al tipo de acompañamiento de la persona al momento del proceso de repatriación y arribo a México.                                                                                                                                 | Int              | 1-5, 9999                              |
| PERM_EU       | Nominal          | Tiempo de permanencia en Estados Unidos.                   | Refiere el tiempo que la persona repatriada declaró haber permanecido en Estados Unidos, en el caso de niños, niñas o adolescentes, corresponde al estado declarado por la autoridad migratoria o el acompañante del NNA.                         | Int              | 1-8, 9999                              |
| EDO_DET       | Nominal          | Estado donde se detuvo al repatriado.                      | Estado de la Unión Americana donde la persona repatriada declaró haber sido detenido, en el caso de niños, niñas o adolescentes, corresponde al estado declarado por la autoridad migratoria o el acompañante de las niñas, niños o adolescentes. | Int              | 1-60, 9999                             |
| AUT_DEP       | Nominal          | Autoridad migratoria que deportó.                          | Hace referencia a la autoridad de Estados Unidos que detuvo a la persona repatriada.                                                                                                                                                              | Int              | 1-4, 9999                              |
| EN_DES        | Nominal          | Entidad federativa de destino.                             | Entidad federativa declarada por la persona repatriada como el lugar al que se dirige.                                                                                                                                                            | Int              | 1-32, 9999                             |
| MUN_DES       | Nominal          | Municipio a donde se dirige el repatriado.                 | Municipio declarado por la persona repatriada como el lugar al que se dirige.                                                                                                                                                                     | Int              | 1001-32058                             |
| CAN_AL        | Nominal          | Canalizado a albergue (lugar para hospedarse y/o asearse). | Indica si la persona repatriada fue canalizada a un albergue (lugar para hospedarse y/o asearse).                                                                                                                                                 | Bool             | 0-1                                    |
| CAN_COM       | Nominal          | Canalizado a comedor.                                      | Indica si la persona repatriada fue canalizada a un comedor.                                                                                                                                                                                      | Bool             | 0-1                                    |
| CAN_DIF       | Nominal          | Canalizado a DIF                                           | Indica si la persona repatriada fue canalizada a un DIF.                                                                                                                                                                                          | Bool             | 0-1                                    |
| CAN_HOS       | Nominal          | Canalizado a hospital (Centro de Salud).                   | Indica si la persona repatriada fue canalizada a un hospital.                                                                                                                                                                                     | Bool             | 0-1                                    |
| CAN_STRA      | Nominal          | Canalizado a la Secretaría del Trabajo.                    | Indica si la persona repatriada fue canalizada al módulo de la Secretaría del Trabajo para orientación laboral.                                                                                                                                   | Bool             | 0-1                                    |
| CAN_SEGPOP    | Nominal          | Canalizado al INSABI.                                      | Indica si la persona repatriada fue canalizada al módulo del Instituto de Salud para el Bienestar (INSABI) para orientación de inscripción.                                                                                                       | Bool             | 0-1                                    |
| CAN_OFAM      | Nominal          | Canalizado a una OFAM.                                     | Indica si la persona repatriada fue canalizada a alguna Oficina de Apoyo al Migrante Estatal para orientación o acceso a servicios (salud, traslado, vinculación laboral, entre otros).                                                           | Bool             | 0-1                                    |
| AGUA_AL       | Nominal          | Se le proporcionó agua o alimento.                         | Indica si a la persona repatriada se le proporcionó agua o alimento durante el proceso de recepción en México.                                                                                                                                    | Bool             | 0-1                                    |
| DESC_BUS      | Nominal          | Se le proporcionó descuento de autobús.                    | Indica si a la persona repatriada se le proporcionó algún descuento en el pasaje de autobús hacia el lugar de destino en México.                                                                                                                  | Bool             | 0-1                                    |
| APO_AUX       | Nominal          | Recibió primeros Auxilios.                                 | Indica si a la persona repatriada se le proporcionaron primeros auxilios durante el proceso de recepción en México.                                                                                                                               | Bool             | 0-1                                    |
| APO_LLAM      | Nominal          | Recibió apoyo en llamada telefónica.                       | Indica si a la persona repatriada se le proporcionó apoyo para realizar una llamada telefónica durante el proceso de recepción en México.                                                                                                         | Bool             | 0-1                                    |
| APO_MAT       | Nominal          | Recibió material informativo.                              | Indica si a la persona repatriada se le proporcionó material informativo durante el proceso de recepción en México.                                                                                                                               | Bool             | 0-1                                    |
| APO_TRANS     | Nominal          | Se le proporcionó traslado local.                          | Indica si a la persona repatriada se le proporcionó traslado local al lugar de destino dentro de la misma ciudad.                                                                                                                                 | Bool             | 0-1                                    |
| APO_VES       | Nominal          | Recibió ropa y/o calzado.                                  | Indica si a la persona repatriada se le proporcionó ropa y calzado durante el proceso de recepción en México.                                                                                                                                     | Bool             | 0-1                                    |
| APO_ACT_NAC   | Nominal          | Facilitación a trámite relacionado con acta de nacimiento. | Indica si a la persona repatriada se le facilitó el apoyo para obtener su acta de nacimiento durante el proceso de recepción en México.                                                                                                           | Bool             | 0-1                                    |
| APO_CURP      | Nominal          | Se le proporcionó apoyo para el trámite de CURP.           | Indica si a la persona repatriada se le facilitó el apoyo para obtener su CURP durante el proceso de recepción en México.                                                                                                                         | Bool             | 0-1                                    |
| APO_ASF       | Nominal          | Se le proporcionó acceso a algún servicio financiero.      | Indica si a la persona repatriada se le facilitó el acceso a algún servicio financiero durante el proceso de recepción en México.                                                                                                                 | Bool             | 0-1                                    |
| APO_AME       | Nominal          | Se le proporcionó acceso a algún módulo educativo.         | Indica si a la persona repatriada se le canalizó al módulo educativo durante el proceso de recepción en México para ofrecerle alternativas educativas.                                                                                            | Bool             | 0-1                                    |
| APO_REC_PERT  | Nominal          | Se le proporcionó apoyo para recuperar sus pertenencias.   | Indica si a la persona repatriada se le apoyó para recuperar sus pertenencias durante el proceso de recepción en México.                                                                                                                          | Bool             | 0-1                                    |
| APO_AT_MEDICA | Nominal          | Se le proporcionó atención médica.                         | Indica si a la persona repatriada se le proporcionó atención médica durante el proceso de recepción en México.                                                                                                                                    | Bool             | 0-1                                    |
| MUN_NAC       | Nominal          | Municipio de nacimiento.                                   | Municipio de nacimiento de la persona repatriada.                                                                                                                                                                                                 | Int              | 1001-32058                             |

Si notamos, la tabla esta codificada a partir de su catálogo. Para poder leerla de una manera más sencilla, realizaremos una serie de left-joins que nos permitirá tener la columna codificada, y la columna descodificada.

Si el nombre de la columna codificada es $x$, la columna descodificada se llamará $x$_parsed. Por ejemplo, "MUN_NAC" y "MUN_NAC_parsed".

El catálogo es un archivo xls, por lo que debemos actualizar la librería que permite leer a pandas este tipo de archivo (de lo contrario nos saldrá un error).

In [26]:
!pip install -I xlrd==1.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached xlrd-1.2.0-py2.py3-none-any.whl (103 kB)


In [27]:
#Leemos primer catalogo
variables = pd.read_excel("data//"+Catalogos_SEGOB_archivo, index_col=0, sheet_name='Variables')
variables.reset_index(drop=True, inplace=True)

#La siguiente linea de código es para quitar el multiindex que tiene la tabla con los catalogos.
variables["Nombre de la variable"]=variables["Nombre de la variable"].fillna(method='ffill')

#Con un ciclo for realizaremos el left merge. Esto lo podemos hacer con todas las columnas que hay que parsear con excepción de "MUN_DES" y "MUN_NAC" 
#que tienen sus propios catalogos y realizaremos el merge aparte.
caracteristicas=SEGOB_tidy.columns.tolist()
caracteristicas.remove('MUN_DES')
caracteristicas.remove('MUN_NAC')

#Realizaremos el merge sobre una copia de la tidy data
SEGOB_tidy_V2=SEGOB_tidy.copy()

In [28]:
for caracteristica in caracteristicas:
    right=variables[variables["Nombre de la variable"]==caracteristica]
    right.rename(columns={ "Valor" : caracteristica, 'Etiqueta' : caracteristica +'_parsed' }, inplace=True)
    right.drop(["Observaciones","Nombre de la variable"], axis=1, inplace=True)

    SEGOB_tidy_V2 = pd.merge(SEGOB_tidy_V2, right, how='left', on=caracteristica)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.p

In [29]:
#Nos falta catalogo "MUN_NAC" y "MUN_DES". Hacemos este último tratamiento a continuación.

mun_nac = pd.read_excel("data//"+Catalogos_SEGOB_archivo, index_col=0, sheet_name='MUN_NAC')
mun_nac.reset_index(drop=True, inplace=True)
mun_des = pd.read_excel("data//"+Catalogos_SEGOB_archivo, index_col=0, sheet_name='MUN_DES')
mun_des.reset_index(drop=True, inplace=True)

for caracteristica in "MUN_NAC", "MUN_DES":
    if(caracteristica == "MUN_NAC"):
        right=mun_nac.copy()
    else:
        right=mun_des.copy()

    right.drop([caracteristica], axis=1, inplace=True)
    right.rename(columns={ "Valor" : caracteristica, 'Etiqueta' : caracteristica +'_parsed' }, inplace=True)

    SEGOB_tidy_V2 = pd.merge(SEGOB_tidy_V2, right, how='left', on=caracteristica)

Finalmente, esta es la tabla descodificada.

In [30]:
SEGOB_tidy_V2

,DEL,FEC_REP,CLASIF_REP,IRE,SEXO,EDA,EN_NAC,NIV_ESC,ACOM_REP,PERM_EU,...,APO_VES_parsed,APO_ACT_NAC_parsed,APO_TRASF_parsed,APO_CURP_parsed,APO_ASF_parsed,APO_AME_parsed,APO_REC_PERT_parsed,APO_AT_MEDICA_parsed,MUN_NAC_parsed,MUN_DES_parsed
0,26,2016-06-06,1,16000001,1,100,14,7,3,1,...,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Puerto Vallarta, Jal."
1,26,2016-01-22,1,16000002,1,99,21,5,3,1,...,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Puebla, Pue."
2,26,2016-11-10,1,16000003,1,100,12,7,3,1,...,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Atenango del Río, Gro."
3,26,2016-07-10,1,16000007,1,88,14,2,3,1,...,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Guadalajara, Jal."
4,26,2016-06-03,1,16000010,1,83,14,2,3,4,...,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Yahualica de González Gallo, Jal."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301218,26,2021-12-31,1,VAOALF11270736640015000000,1,36,15,3,3,1,...,No,No,NaN,No,No,No,No,No,"Nezahualcóyotl, Mex.",No especificado
301219,26,2021-12-31,1,VEHERIA1130648896001100000,1,25,11,7,3,1,...,No,No,NaN,No,No,No,No,No,"Pénjamo, Gto.",No especificado
301220,26,2021-12-31,1,CUCJUAE1130670496001100000,1,25,11,3,3,1,...,No,No,NaN,No,No,No,No,No,"Abasolo, Gto.",No especificado
301221,26,2021-12-31,1,ZAQREB21306316160020000000,2,25,20,5,4,1,...,No,No,NaN,No,No,No,No,No,"Santos Reyes Nopala, Oax.",No especificado
